# Summary analysis
Mouna Ammar & Paul Gazel

Bla bla bla introduction

In [5]:
from pymongo import MongoClient
from bson.son import SON
import pymongo

from datetime import datetime

In [6]:
client = MongoClient(host='localhost', port=27017, maxPoolSize=200)
print(client.list_database_names())

covidDB = client['Covid_db22'].covid_data22

['Covid_db22', 'admin', 'config', 'local']


In [7]:
covidDB.count_documents({})

43229

Comments on correctly importing -> only 43229 docs not same as summary explained because when inserting 2897 where considered duplicates. 46126 - 2897 = 43229

Introduction to what we will do

In [8]:
covidDB.find_one()

{'_id': ObjectId('61c20d9f913096a49a9e2b5b'),
 'source': {'id': 'nbc-news', 'name': 'NBC News'},
 'author': 'Ilene Prusher',
 'title': "The best gifts don't come in shiny boxes",
 'description': 'This holiday season we have an opportunity to extend our time together beyond watching someone’s face as they tear the wrapping off their booty.',
 'publishedAt': '2021-12-12T10:30:18Z',
 'content': 'Supply chains are all gummed up, leaving some shelves as empty as the toilet paper aisles of March 2020. Prices rose by an alarming 6.8 percent last month, the highest annual inflation rate in almost… [+5211 chars]'}

In [ ]:
# Start by plotting over time

pipeline = [
    { '$group': {
    '_id': {'$dateToString': { 'date': { '$toDate': '$publishedAt' }, 'format': '%Y-%m-%d' } },
    'count': { '$sum': 1 }
  }},
    {'$sort': SON([('_id', pymongo.ASCENDING)])}
]

res = covidDB.aggregate(pipeline)

x = []
y = []
for sample in res:
    x.append(datetime.strptime(sample['_id'], '%Y-%m-%d'))
    y.append(sample['count'])
    

fig = go.Figure([go.Scatter(x=x, y=y)])
fig.update_layout(
    title="Number of articles over time",
    xaxis_title="Time (in days)",
    yaxis_title="Number of articles"
)
fig.show()

In [ ]:
# Look for most repeated numbers for total and then each category/language/country (or the most relevant one)

In [53]:
from collections import Counter
from nltk.tokenize import sent_tokenize, word_tokenize

import nltk
from nltk.corpus import stopwords

def get_counter_from_query(query, attribute):
    my_counter = Counter()
    
    for obj in covidDB.find(query):
        if obj[attribute]:
            # Tokenize and give it to the counter
            tokens = word_tokenize(obj[attribute].lower())
            my_counter.update(tokens)
    
    # remove stop words
    stops = set(stopwords.words('english'))
    for word in stops:
        if word in my_counter:
            del my_counter[word]
    
    # Remove items under size 2 chars
    del_list =[]
    for key in my_counter:
        if len(key) <= 2:
            del_list.append(key)

    for item in del_list:
        del my_counter[item]
    
    return my_counter




need_translation = ['aftenposten', 'ansa', 'argaam', 'ary-news', 'bild', 'blasting-news-br', 'cnn-es', 'der-tagesspiegel', 'die-zeit', 'el-mundo', 'focus', 'globo', 'google-news-ar', 'google-news-br', 'google-news-fr', 'google-news-is', 'google-news-it', 'google-news-ru', 'google-news-sa', 'goteborgs-posten', 'gruenderszene', 'handelsblatt', 'il-sole-24-ore', 'infobae', 'info-money', 'la-gaceta', 'la-nacion', 'la-repubblica', 'le-monde', 'lenta', 'lequipe', 'les-echos', 'liberation', 'marca', 'nrk', 'rbc', 'rt', 'rtl-nieuws', 'sabq', 'spiegel-online', 'svenska-dagbladet', 't3n', 'techcrunch-cn', 'wired-de', 'wirtschafts-woche', 'xinhua-net', 'ynet']
test = {'source.id': {'$nin': need_translation}}
att = 'title'

count = get_counter_from_query(test, att)

In [55]:
count.most_common(20)

[('covid', 4042),
 ('covid-19', 3859),
 ('omicron', 2722),
 ('new', 2414),
 ('reuters', 2297),
 ('vaccine', 1772),
 ('variant', 1628),
 ('says', 1612),
 ('cases', 1396),
 ('biden', 1164),
 ('news24.com', 736),
 ('travel', 675),
 ('amid', 631),
 ('india', 620),
 ('health', 616),
 ('pandemic', 592),
 ('south', 571),
 ('first', 566),
 ('africa', 559),
 ('booster', 553)]

In [61]:
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree

def get_name_entities(text):
    chunked = ne_chunk(pos_tag(word_tokenize(text)))
    continuous_chunk = []
    current_chunk = []
    
    for i in chunked:
        if type(i) == Tree:
            current_chunk.append(" ".join([token for token, pos in i.leaves()]))
        if current_chunk:
            named_entity = " ".join(current_chunk)
            continuous_chunk.append(named_entity)
            current_chunk = []
        else:
            continue
    return continuous_chunk

def get_counter_name_entities_from_query(query, attribute):
    my_counter = Counter()
    
    for obj in covidDB.find(query):
        if obj[attribute]:
            # Tokenize and give it to the counter
            name_entities = get_name_entities(obj[attribute].lower())
            my_counter.update([x.lower() for x in name_entities])
    
    return my_counter


In [62]:
need_translation = ['aftenposten', 'ansa', 'argaam', 'ary-news', 'bild', 'blasting-news-br', 'cnn-es', 'der-tagesspiegel', 'die-zeit', 'el-mundo', 'focus', 'globo', 'google-news-ar', 'google-news-br', 'google-news-fr', 'google-news-is', 'google-news-it', 'google-news-ru', 'google-news-sa', 'goteborgs-posten', 'gruenderszene', 'handelsblatt', 'il-sole-24-ore', 'infobae', 'info-money', 'la-gaceta', 'la-nacion', 'la-repubblica', 'le-monde', 'lenta', 'lequipe', 'les-echos', 'liberation', 'marca', 'nrk', 'rbc', 'rt', 'rtl-nieuws', 'sabq', 'spiegel-online', 'svenska-dagbladet', 't3n', 'techcrunch-cn', 'wired-de', 'wirtschafts-woche', 'xinhua-net', 'ynet']
test = {'source.id': {'$nin': need_translation}}
att = 'title'

count = get_counter_name_entities_from_query(test, att)

In [63]:
count.most_common(20)

[('u.s.', 533),
 ('u.s', 21),
 ('covid-19', 15),
 ('u.n.', 10),
 ('u.s.-china', 5),
 ('china', 4),
 ('d.c.', 3),
 ('न', 2),
 ('russia', 2),
 ('pfizer/biontech', 2),
 ('st.', 2),
 ('brother-in-law', 2),
 ('l.a.', 2),
 ('china-u.s.', 2),
 ('on-site', 2),
 ("'disgraced", 1),
 ('u.k.', 1),
 ('इन', 1),
 ('u.s.-canada', 1),
 ('north-south', 1)]